<h1>
<center>
Module 5: Bias-Variance and Cross-Validation
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
with open('/content/gdrive/My Drive/class_tables/loan_wrangled_week3.csv', 'r') as f:
  loan_table = pd.read_csv(f)

loan_table.head(2)  #make sure it looks ok - we see the results of our week 2 wrangling

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,...,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,...,0,0,0,1,0,low,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,...,0,0,0,1,0,low,1,0,0,0


In [0]:
pd.set_option('display.max_columns', None)

<h2>A couple of notes</h2>
I added points to assignments to give you an idea of how much of a challenge I think they are. Let me know if I am off base.
<p>
  Don't load your library yet. Finish problem 1 before importing your library.

<hr>
<h1>
1: Fix our scoring functions (30 points)
</h1>
<p>
<div class=h1_cell>
<p>
As we defined our scoring functions in week 2, they all assume that the 4 cases have values greater than 0. But this week we will see that is not always the case. And hence you will get an error like this if you try using the scoring functions as is:
  <pre>
  KeyError: 'false_negative'
  </pre>
<p>
What this means is that there were no false negatives found so the key is also not found. You would find the problem in a statement like this:
  <pre>
  fn = cases['false_negative']  #key does not exist because no false negatives
  </pre>
  Your task is to fix up the 3 scoring functions so they handle cases like this. And while you are at it, make sure you never divide by 0. Once you have them working, you can add them to your library for this week. I give you some test cases to code against.
  <p>
 Important: do not import your library before (a) you have all functions below working correctly, and (b) you have replaced the old functions in your library with the new ones.
</div>

In [0]:
def accuracy(cases):
  tp = 0
  if 'true_positive' in cases:
    tp = cases['true_positive']
  tn = 0
  if 'true_negative' in cases:
    tn = cases['true_negative']
  fp = 0
  if 'false_positive' in cases:
    fp = cases['false_positive']
  fn = 0
  if 'false_negative' in cases:
    fn = cases['false_negative']

  if(tp+tn+fp+fn) == 0:
    return 0
  
  return (tp + tn)/(tp+tn+fp+fn)
  


In [5]:
#test

print(('should be 0', accuracy(pd.Series({}))))
print(('should be 0.3', accuracy(pd.Series({'true_positive':1, 'true_negative':2, 'false_positive':3, 'false_negative':4}))))
print(('should be 0.2', accuracy(pd.Series({'true_negative':2, 'false_positive':3, 'false_negative':4}))))

('should be 0', 0)
('should be 0.3', 0.3)
('should be 0.2', 0.2222222222222222)


In [0]:
def f1(cases):
  #the heart of the matrix
  tp = 0
  if 'true_positive' in cases:
    tp = cases['true_positive']
  tn = 0
  if 'true_negative' in cases:
    tn = cases['true_negative']
  fp = 0
  if 'false_positive' in cases:
    fp = cases['false_positive']
  fn = 0
  if 'false_negative' in cases:
    fn = cases['false_negative']

    #other measures we can derive
  if (((tp+fn) == 0) or ((tp+fp) == 0)):
    return 0
  else:
    recall = 1.0*tp/(tp+fn)
    precision = 1.0*tp/(tp+fp)

    #now for the one we want
  if ((recall == 0) or (precision == 0)):
    return 0
  else:
       f1 = 2/(1/recall + 1/precision)

  return f1


In [7]:
#test

print(('should be 0', f1(pd.Series({}))))
print(('should be 0.2', f1(pd.Series({'true_positive':1, 'true_negative':2, 'false_positive':3, 'false_negative':4}))))
print(('should be 0', f1(pd.Series({'true_negative':2, 'false_positive':3, 'false_negative':4}))))

('should be 0', 0)
('should be 0.2', 0.2222222222222222)
('should be 0', 0)


In [0]:
def informedness(cases):
  tp = 0
  if 'true_positive' in cases:
    tp = cases['true_positive']
  tn = 0
  if 'true_negative' in cases:
    tn = cases['true_negative']
  fp = 0
  if 'false_positive' in cases:
    fp = cases['false_positive']
  fn = 0
  if 'false_negative' in cases:
    fn = cases['false_negative']
  if (((tp+fn) == 0) or ((tn+fp) == 0)):
    return -1
  else:
    recall = 1.0*tp/(tp+fn)
    specificity = 1.0*tn/(tn+fp)
    J = (recall + specificity) - 1

  return J


In [9]:
#test

print(('should be -1', informedness(pd.Series({}))))
print(('should be -0.39', informedness(pd.Series({'true_positive':1, 'true_negative':2, 'false_positive':3, 'false_negative':4}))))
print(('should be -0.6', informedness(pd.Series({'true_negative':2, 'false_positive':3, 'false_negative':4}))))

('should be -1', -1)
('should be -0.39', -0.3999999999999999)
('should be -0.6', -0.6)


<h1>Ok, now update your library</h1>
When you have done that, you are ready to import.
<p>
  <b>Important note</b>: you may be tempted to skip the step of updating your library before importing. Why do that when you already have the new functions defined above? We are back to that static-scope thing. Your notebook context (namespace) can see your library functions but your library functions cannot see your notebook functions. The library has its own namespace. In summary, `produce_scores` will not see your functions above. Instead it will see the functions in your library. And if you don't update your library, you will be using your old, broken versions. And it does not matter if you import first then define new functions in the notebook. Your notebook functions still will not override your library functions. Whew. So go update your library!

In [10]:
!rm library_w19_week5.py

rm: cannot remove 'library_w19_week5.py': No such file or directory


In [11]:
from google.colab import files
files.upload()

Saving library_w19_week5.py to library_w19_week5.py


{'library_w19_week5.py': b'import pandas as pd\nimport numpy as np\nfrom functools import reduce\nfrom types import SimpleNamespace\n\ndef predictor_case(row, pred, target):\n\tcase_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n\tactual = row[target]\n\tprediction = row[pred]\n\tcase = case_dict[(prediction, actual)]\n\treturn case\n\ndef accuracy(cases):\n\n    tp = 0\n    if \'true_positive\' in cases:\n        tp = cases[\'true_positive\']\n    tn = 0\n    if \'true_negative\' in cases:\n        tn = cases[\'true_negative\']\n    fp = 0\n    if \'false_positive\' in cases:\n        fp = cases[\'false_positive\']\n    fn = 0\n    if \'false_negative\' in cases:\n        fn = cases[\'false_negative\']\n\n    if(tp+tn+fp+fn) == 0:\n        return 0\n\n    return (tp + tn)/(tp+tn+fp+fn)\n\n#accuracy(p1_types)\n\ndef f1(cases):\n\n    #the heart of the matrix\n    #the heart of the matrix\n    tp = 0\n    if \'true_posi

In [12]:
from library_w19_week5 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 f1	 find_best_splitter	 generate_table	 gig	 
gini	 informedness	 k_fold	 path_id	 predictor_case	 probabilities	 produce_scores	 reorder_paths	 tree_predictor	 



<hr>
<h1>
2: Explore depth parameter with 10-folding (10 points)
</h1>
<p>
<div class=h1_cell>
<p>
In the week 4 module you were asked to explore depths from 1 to 5 without any cross-validation. Go ahead and do it again but using K-folding with K=10.
<p>
First, define the columns to use. I do that for you below.
</div>

In [0]:
splitter_columns = [
        #ApplicantIncome
       'apin_low', 'apin_high', 'apin_average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'lam_Low', 'lam_Average', 'lam_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
Show your results below
</h2>
<p>
<div class=h1_cell>
<p>
Explore depths 1-5 with 10-folding. You can leave the gig cutoff as default value.
</div>

In [14]:
table10_1 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth': 1}, splitter_columns)
table10_1

,name,accuracy,f1,informedness
0,fold_0,0.770492,0.847826,0.363636
1,fold_1,0.852459,0.905263,0.500000
2,fold_2,0.754098,0.835165,0.330952
3,fold_3,0.770492,0.860000,0.212567
4,fold_4,0.803279,0.875000,0.365633
5,fold_5,0.803279,0.877551,0.330214
6,fold_6,0.836066,0.886364,0.545455
7,fold_7,0.868852,0.914894,0.565508
8,fold_8,0.786885,0.865979,0.310078
9,fold_9,0.846154,0.895833,0.545455


In [15]:
table10_2 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth': 2}, splitter_columns)
table10_2

,name,accuracy,f1,informedness
0,fold_0,0.770492,0.847826,0.363636
1,fold_1,0.852459,0.905263,0.500000
2,fold_2,0.754098,0.835165,0.330952
3,fold_3,0.770492,0.860000,0.212567
4,fold_4,0.803279,0.875000,0.365633
5,fold_5,0.803279,0.877551,0.330214
6,fold_6,0.819672,0.876404,0.500000
7,fold_7,0.868852,0.914894,0.565508
8,fold_8,0.786885,0.865979,0.310078
9,fold_9,0.846154,0.895833,0.545455


In [16]:
table10_3 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth': 3}, splitter_columns)
table10_3

,name,accuracy,f1,informedness
0,fold_0,0.770492,0.847826,0.363636
1,fold_1,0.852459,0.905263,0.500000
2,fold_2,0.721311,0.808989,0.280952
3,fold_3,0.770492,0.860000,0.212567
4,fold_4,0.803279,0.875000,0.365633
5,fold_5,0.803279,0.877551,0.330214
6,fold_6,0.819672,0.876404,0.500000
7,fold_7,0.868852,0.914894,0.565508
8,fold_8,0.786885,0.865979,0.310078
9,fold_9,0.846154,0.895833,0.545455


In [17]:
table10_4 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth': 4}, splitter_columns)
table10_4

,name,accuracy,f1,informedness
0,fold_0,0.770492,0.847826,0.363636
1,fold_1,0.852459,0.905263,0.500000
2,fold_2,0.737705,0.822222,0.305952
3,fold_3,0.770492,0.860000,0.212567
4,fold_4,0.803279,0.875000,0.365633
5,fold_5,0.803279,0.875000,0.366310
6,fold_6,0.819672,0.876404,0.500000
7,fold_7,0.868852,0.914894,0.565508
8,fold_8,0.786885,0.865979,0.310078
9,fold_9,0.846154,0.895833,0.545455


In [18]:
table10_5 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth': 5}, splitter_columns)
table10_5

,name,accuracy,f1,informedness
0,fold_0,0.770492,0.847826,0.363636
1,fold_1,0.852459,0.905263,0.500000
2,fold_2,0.754098,0.835165,0.330952
3,fold_3,0.770492,0.860000,0.212567
4,fold_4,0.803279,0.875000,0.365633
5,fold_5,0.803279,0.875000,0.366310
6,fold_6,0.836066,0.886364,0.545455
7,fold_7,0.868852,0.914894,0.565508
8,fold_8,0.754098,0.842105,0.263566
9,fold_9,0.846154,0.895833,0.545455


<hr>
<h1>
What does it look like to you?
</h1>
<p>
<div class=h1_cell>
<p>
To me it looks like the stump is best! As we add depth I don't see an improvement. And it looks like results from depth 4 and 5 the same. What I would try next is the gig cutoff and bringing more columns into play (e.g., Gender, Married) by applying ohe to them.
</div>

<hr>
<h1>
4: Random slices (60 points)
</h1>
<p>
<div class=h1_cell>
<p>
I'd like you to try one more thing. The `k_fold` function takes slices sequentially. I'd like you to write a function `k_fold_random` that takes random slices *without replacement*. Without replacement means that each slice should have a unique set of rows, i.e., no slices share a row
<p>
Each slice should be roughly 1/k percent of the entire table.
<p>
To solve this problem, I found it easiest to play around with loops in a notebook cell before trying to replace code in `k_fold`. My target was the equivalent of this but now with random slices:
<pre>
<code>
    total_len = len(table.index)
    slice_size = int(total_len/(1.0`*`k))
    slices = []

    #generate the slices
    for i in range(k-1):
        a_slice =  table[i`*`slice_size:(i+1)`*`slice_size]
        slices.append( a_slice )
    slices.append( table[(k-1)`*`slice_size:] )
    
    verify_unique(slices)  # I ask you to define this debugging function below
</code>
</pre>
<p>
I copied this code into a cell and then started to play around with various ideas until I got one I liked.
<p>
</div>

<h2>Help with debugging</h2>
I'll define a debugging function that will take a list of slices and do a pairwise comparison. For each pair, it will print the set intersection between the 2 slices. I give you a few test cases to try.

In [0]:
#Determine if slices are mutually exclusive
def verify_unique(slices):
    print(('total length all slices', sum([len(s) for s in slices])))
    for i, a_slice in enumerate(slices[:-1]):
        a_set = set(a_slice.index)
        for j, b_slice in enumerate(slices[i+1:]):
            b_set = set(b_slice.index)
            int_set = a_set.intersection(b_set)  # should be empty set as result
            print((i,j+i+1,int_set))
    return None

In [20]:
#test

verify_unique([loan_table[0:2], loan_table[2:4], loan_table[4:5]])  #looks good - enpty set for each pair

('total length all slices', 5)
(0, 1, set())
(0, 2, set())
(1, 2, set())


In [21]:
#test

verify_unique([loan_table[0:2], loan_table[1:4], loan_table[1:5]])  #not so good, finds intersection between all 3 pairs

('total length all slices', 9)
(0, 1, {1})
(0, 2, {1})
(1, 2, {1, 2, 3})


In [24]:
# here is sequential slice code from k_fold if you want to use it as base.
# modify it to produce slices with random rows in each slice.
from sklearn.utils import shuffle


table = shuffle(loan_table)
k = 10

total_len = len(table.index)
split_size = int(total_len/(1.0*k))
slices = []

#generate the slices
for i in range(k-1):
    a_slice =  table[i*split_size:(i+1)*split_size]
    slices.append( a_slice )
slices.append( table[(k-1)*split_size:] )

verify_unique(slices)  # should see 614 length and empty sets all the way down


('total length all slices', 614)
(0, 1, set())
(0, 2, set())
(0, 3, set())
(0, 4, set())
(0, 5, set())
(0, 6, set())
(0, 7, set())
(0, 8, set())
(0, 9, set())
(1, 2, set())
(1, 3, set())
(1, 4, set())
(1, 5, set())
(1, 6, set())
(1, 7, set())
(1, 8, set())
(1, 9, set())
(2, 3, set())
(2, 4, set())
(2, 5, set())
(2, 6, set())
(2, 7, set())
(2, 8, set())
(2, 9, set())
(3, 4, set())
(3, 5, set())
(3, 6, set())
(3, 7, set())
(3, 8, set())
(3, 9, set())
(4, 5, set())
(4, 6, set())
(4, 7, set())
(4, 8, set())
(4, 9, set())
(5, 6, set())
(5, 7, set())
(5, 8, set())
(5, 9, set())
(6, 7, set())
(6, 8, set())
(6, 9, set())
(7, 8, set())
(7, 9, set())
(8, 9, set())


<h2>
Plug your code in
</h2>
<p>
<div class=h1_cell>
<p>
Once you are generating slices randomly, you will have k test slices. You can go ahead and place your code into `k_fold_random`. You should now be good to go.
<p>
One problem with doing random selections is that we will get different results every time we run our code. We will see a way to get repeatable results in next module. For now, you may find that you are not matching my fold results.
</div>

In [0]:
def k_fold_random(table, k, target, hypers, candidate_columns):
  #set up the table where we will record fold results
  result_columns = ['name',  'accuracy', 'f1', 'informedness']
  k_fold_results_table = pd.DataFrame(columns=result_columns)

  #generate the slices
  table = shuffle(loan_table)
  total_len = len(table.index)
  split_size = int(total_len/(1.0*k))
  slices = []

  #generate the slices
  for i in range(k-1):
    a_slice =  table[i*split_size:(i+1)*split_size]
    slices.append( a_slice )
  slices.append( table[(k-1)*split_size:] )

  verify_unique(slices)  # should see 614 length and empty sets all the way down

  #generate test results
  all_scores = []  #keep track of all k results
  for i in range(k):
      test_table = slices[i]
      train_table = compute_training(slices, i)
      fold_tree = build_tree_iter(train_table, candidate_columns, target, hypers)  # train
      scores = produce_scores(test_table, fold_tree, target)  # test
      results_row = {'name': 'fold_'+str(i), 'accuracy': scores[0], 'f1': scores[1], 'informedness': scores[2]}
      k_fold_results_table = k_fold_results_table.append(results_row,ignore_index=True)
      all_scores.append(scores)

  #compute average of all folds
  avg_scores = tuple(reduce(lambda total, triple: np.add(triple, total), all_scores)/k)
  results_row = {'name': 'average', 'accuracy': avg_scores[0], 'f1': avg_scores[1], 'informedness': avg_scores[2]}
  k_fold_results_table = k_fold_results_table.append(results_row,ignore_index=True)

  #note that I add the meta comment as last step to avoid it being wiped out
  k_fold_results_table.meta = SimpleNamespace()
  k_fold_results_table.meta.hypers  = hypers # adds comment to remind me of hyper params used

  return k_fold_results_table




In [32]:
k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth': 1}, splitter_columns)


('total length all slices', 614)
(0, 1, set())
(0, 2, set())
(0, 3, set())
(0, 4, set())
(0, 5, set())
(0, 6, set())
(0, 7, set())
(0, 8, set())
(0, 9, set())
(1, 2, set())
(1, 3, set())
(1, 4, set())
(1, 5, set())
(1, 6, set())
(1, 7, set())
(1, 8, set())
(1, 9, set())
(2, 3, set())
(2, 4, set())
(2, 5, set())
(2, 6, set())
(2, 7, set())
(2, 8, set())
(2, 9, set())
(3, 4, set())
(3, 5, set())
(3, 6, set())
(3, 7, set())
(3, 8, set())
(3, 9, set())
(4, 5, set())
(4, 6, set())
(4, 7, set())
(4, 8, set())
(4, 9, set())
(5, 6, set())
(5, 7, set())
(5, 8, set())
(5, 9, set())
(6, 7, set())
(6, 8, set())
(6, 9, set())
(7, 8, set())
(7, 9, set())
(8, 9, set())


,name,accuracy,f1,informedness
0,fold_0,0.885246,0.927835,0.562500
1,fold_1,0.819672,0.891089,0.311594
2,fold_2,0.770492,0.851064,0.333333
3,fold_3,0.918033,0.952381,0.545455
4,fold_4,0.704918,0.800000,0.251716
5,fold_5,0.868852,0.911111,0.607769
6,fold_6,0.836066,0.883721,0.565217
7,fold_7,0.704918,0.800000,0.264640
8,fold_8,0.754098,0.831461,0.364989
9,fold_9,0.830769,0.895238,0.390931


What we got with prior sequential split at depth 1: 
<pre>
10	average	0.809206	0.876388	0.406950
</pre>

In [33]:
k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth': 2}, splitter_columns)


('total length all slices', 614)
(0, 1, set())
(0, 2, set())
(0, 3, set())
(0, 4, set())
(0, 5, set())
(0, 6, set())
(0, 7, set())
(0, 8, set())
(0, 9, set())
(1, 2, set())
(1, 3, set())
(1, 4, set())
(1, 5, set())
(1, 6, set())
(1, 7, set())
(1, 8, set())
(1, 9, set())
(2, 3, set())
(2, 4, set())
(2, 5, set())
(2, 6, set())
(2, 7, set())
(2, 8, set())
(2, 9, set())
(3, 4, set())
(3, 5, set())
(3, 6, set())
(3, 7, set())
(3, 8, set())
(3, 9, set())
(4, 5, set())
(4, 6, set())
(4, 7, set())
(4, 8, set())
(4, 9, set())
(5, 6, set())
(5, 7, set())
(5, 8, set())
(5, 9, set())
(6, 7, set())
(6, 8, set())
(6, 9, set())
(7, 8, set())
(7, 9, set())
(8, 9, set())


,name,accuracy,f1,informedness
0,fold_0,0.672131,0.767442,0.250549
1,fold_1,0.819672,0.891089,0.314590
2,fold_2,0.688525,0.781609,0.296296
3,fold_3,0.901639,0.936170,0.665278
4,fold_4,0.770492,0.844444,0.391304
5,fold_5,0.868852,0.916667,0.529412
6,fold_6,0.901639,0.938776,0.621581
7,fold_7,0.754098,0.845361,0.239348
8,fold_8,0.836066,0.897959,0.411765
9,fold_9,0.861538,0.910891,0.526316


What we got in sequential at depth 2:
<pre>
10	average	0.807566	0.875392	0.402404
</pre>
Let's try one more.

In [34]:
k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth': 3}, splitter_columns)

('total length all slices', 614)
(0, 1, set())
(0, 2, set())
(0, 3, set())
(0, 4, set())
(0, 5, set())
(0, 6, set())
(0, 7, set())
(0, 8, set())
(0, 9, set())
(1, 2, set())
(1, 3, set())
(1, 4, set())
(1, 5, set())
(1, 6, set())
(1, 7, set())
(1, 8, set())
(1, 9, set())
(2, 3, set())
(2, 4, set())
(2, 5, set())
(2, 6, set())
(2, 7, set())
(2, 8, set())
(2, 9, set())
(3, 4, set())
(3, 5, set())
(3, 6, set())
(3, 7, set())
(3, 8, set())
(3, 9, set())
(4, 5, set())
(4, 6, set())
(4, 7, set())
(4, 8, set())
(4, 9, set())
(5, 6, set())
(5, 7, set())
(5, 8, set())
(5, 9, set())
(6, 7, set())
(6, 8, set())
(6, 9, set())
(7, 8, set())
(7, 9, set())
(8, 9, set())


,name,accuracy,f1,informedness
0,fold_0,0.786885,0.860215,0.380952
1,fold_1,0.868852,0.914894,0.555556
2,fold_2,0.819672,0.884211,0.433333
3,fold_3,0.803279,0.882353,0.244928
4,fold_4,0.819672,0.881720,0.449875
5,fold_5,0.754098,0.838710,0.318182
6,fold_6,0.852459,0.898876,0.571429
7,fold_7,0.819672,0.884211,0.425134
8,fold_8,0.704918,0.804348,0.221445
9,fold_9,0.815385,0.880000,0.428571


What we got in sequential at depth 3:
<pre>
10	average	0.804288	0.872774	0.397404
</pre>
Random looks very close to sequential. With this small a dataset, I could see that happening. In general, random is prefered. With sequential you might be picking up some funny ordering of the rows. Maybe they listed the best applicants first.

<hr>
<h1>Did not change loan table</h1>
<div class=h1_cell>
<p>
No need to write it out.
</div>

<hr>
<h1>We did change library</h1>
<div class=h1_cell>
<p>
We added `verify_unique` and `k_fold_random`. Add them to your library and call new file `library_w19_5b.py`. I added the `b` to differentiate it from functions we added in intro notebook and top of this one.
</div>